# **What is data preprocessing?**

Data preprocessing is a data mining technique that involves transforming raw data into an understandable format.


**Load the library to run the functions**





In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris
from scipy.stats import skew
from numpy import array 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor 
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
dataset = pd.DataFrame()


X = pd.DataFrame()
y = pd.DataFrame()
dataset_minmax_pca = pd.DataFrame()
dataset_minmax = pd.DataFrame()
dataset_minmax_corr = pd.DataFrame()

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

## Load the dataset

In [9]:
def loaddata(file):
    dataset =  pd.read_csv(file)
    dataset.head(5)
    dataset = pd.DataFrame(dataset).fillna(0)
#   dataset = pd.get_dummies(dataset,columns= ['attack_cat'])
    X=dataset.drop(columns=['label'], axis = 1)
    X1=X
    print(kurtosis(X))
    print(print("Skewness for data :",skew(X1)))
    y=dataset['label']
    return dataset,X,y

# 1.**Correlation analysis**




*   Correlation can help in predicting one attribute from another (Great way to impute missing values).
*   List itemCorrelation can (sometimes) indicate the presence of a causal relationship.




**X (train) , y(label)**

**Remove the highly correlated features : to each other**

Advantage: Reduces noise 

[Reference](https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/)

### 1.1**Function to remove highly correlated data**

In [10]:
def removeHcorrFeature(percentage,X):
  # Selecting all the features with high correlation values with other features
    corr_matrix = X.corr().abs()

  # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

  # Find index of feature columns with correlation greater than 0.9
    o_drop = [column for column in upper.columns if any(upper[column] > percentage)]
    print(o_drop)
    return o_drop

# 2.**Feature Scaling**

### 2.1 **MinMaxScaling**

About MinMax Scaler : 
* This Scaler responds well if the standard deviation is small and when a distribution is not Gaussian.     
* This Scaler is sensitive to outliers.

**Function for min max scaling**

In [11]:
def applyMinMax(data):  
# scale features
    scaler = MinMaxScaler()
    model=scaler.fit(data)
    scaled_data=model.transform(data)
    return scaled_data
  #print(data_set)

### 2.2 **Standard Scaling**

About Standard Scaler: 
* Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. 
* This scaler responds well when a distribution is Gaussian.

**Function for standard scaler** **bold text**

In [12]:
def applyStandardScaler(data):
    scaler = StandardScaler()
    model = scaler.fit(data)
    scaled_data = model.transform(data)
  #print(data_set)
    return scaled_data

## Which scaler to use?

This we be decide after we study our dataset for gaussian distribution.

# 3.**Principle component Analysis**

### 3.1**Functions for applying PCA method**

In [13]:
def applyPCA(X, n_com):
   #print(data_set)
    pca = PCA(n_components=n_com).fit(X)
    X_train_pca = pca.transform(X)
    print(pca.n_components_)
    return X_train_pca

# Main function 

Points to note:

1) Scaling is critical while performing Principal Component Analysis(PCA). PCA tries to get the features with maximum variance, and the variance is high for high magnitude features and skews the PCA towards high magnitude features. [Reference](https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35)     

In [14]:
def without_preprocess(filename):
    dataset,X,y=loaddata(filename)
    
    return X,y,dataset    
    #check if the dataset is gaussian

In [15]:
X,y,dataset=without_preprocess("./input/cleaned_dataset_label_encoding.csv")

[ 6.80401589e+01  1.41202499e+01  3.25489325e+00  2.24573259e+00
  2.21322594e+03  2.96582425e+03  2.61644857e+03  3.27228347e+03
  1.49798961e+01 -1.49491386e+00 -1.33661251e+00  1.36968810e+02
  2.27856540e+01  2.55141952e+03  3.50271171e+03  6.73801510e+01
  9.39548143e+02  3.14482186e+02  2.63843613e+03 -1.99181834e+00
 -4.33890762e-01 -4.21084764e-01 -1.98316474e+00  1.58251093e+02
  3.98225846e+02  2.04622331e+02  1.36950231e+01  8.26080496e+00
  3.79268744e+04  8.51365167e+03  1.85341547e+00  5.06444942e+00
  5.09737732e+00  5.68070487e+00  2.95083453e+00  2.07832178e+00
  7.44189174e+01  1.52188534e+02  6.95250182e+02  4.08548752e+00
  1.87480693e+00  6.50723925e+01 -1.56126234e-01]
Skewness for data : [ 8.02244778e+00 -3.79926482e+00  1.78400999e+00 -8.54334812e-01
  4.25216505e+01  4.11925360e+01  4.79168946e+01  4.43399752e+01
  3.38046837e+00 -6.82162799e-01  7.87244421e-01  8.93325379e+00
  4.71074517e+00  4.71204072e+01  4.60860573e+01  8.28790196e+00
  2.69651543e+01  1.

# Check Dataset for Gaussian Distribution
### To decide which scaler to use

In [16]:
def plot_GD(dataset):
  num_features = dataset.columns[dataset.dtypes != 'object']
  for i in num_features:
    plt.figure()
    sns.distplot(dataset[i])
plot_GD(dataset)

NameError: name 'sns' is not defined

<Figure size 432x288 with 0 Axes>

# Conclusion:
Since most of the non object data is noramilzed, We would go ahead with standard scaler. 

# 6.**Save the preprocessed file**